# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=7


# Trade profile

Choose country and years of interest

In [15]:
country_of_interest = comtrade.m49_angola
years_of_interest = '2010,2015,2018,2019,2020,2021'  #comtrade.year_range(2011, 2021)   
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Angola 2010,2015,2018,2019,2020,2021


### Global exports and imports

In [5]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format


global_trade = comtrade.get_global_stats(country_of_interest, years=years_of_interest,
                                         exports_from_word_imports=True)


/Users/jrc/develop/cipf-comtrade/comtrade.py:369: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")
/Users/jrc/develop/cipf-comtrade/comtrade.py:369: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [6]:
cols = ['refYear','flowDesc','countryDesc','reporterDesc','primaryValueFormated']
global_trade[cols]

,refYear,flowDesc,countryDesc,reporterDesc,primaryValueFormated
0,2010,Imports,Angola,Angola,"18,143,268,908.00"
0,2010,Exports,Angola,World,"53,459,185,324.68"
1,2015,Imports,Angola,Angola,"21,549,256,530.51"
1,2015,Exports,Angola,World,"36,829,899,264.38"
2,2018,Imports,Angola,Angola,"16,006,910,579.06"
2,2018,Exports,Angola,World,"44,183,954,089.23"
3,2019,Imports,Angola,Angola,"13,961,640,425.76"
3,2019,Exports,Angola,World,"37,541,664,634.40"
4,2020,Imports,Angola,Angola,"9,337,925,708.64"
4,2020,Exports,Angola,World,"23,994,770,038.40"


## Principais parceiros comerciais
### Principais destinos das exportações

Usamos as importações declaradas pelos parceiros para obter as
exportações do país de interesse.

In [24]:
show_ony_top=10
years_of_interest = comtrade.year_range(2011, 2022)

top_partners = comtrade.top_partners(reporterCode=None,partnerCode=country_of_interest,years=years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)
top_partners.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1189 entries, 23 to 3224
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  1189 non-null   object 
 1   freqCode                  1189 non-null   object 
 2   refPeriodId               1189 non-null   int64  
 3   refYear                   1189 non-null   int64  
 4   refMonth                  1189 non-null   int64  
 5   period                    1189 non-null   object 
 6   reporterCode              1189 non-null   int64  
 7   reporterISO               0 non-null      object 
 8   reporterDesc              1189 non-null   object 
 9   flowCode                  1189 non-null   object 
 10  flowDesc                  1189 non-null   object 
 11  partnerCode               1189 non-null   int64  
 12  partnerISO                0 non-null      object 
 13  partnerDesc               1189 non-null   object 
 14  partner

/Users/jrc/develop/cipf-comtrade/comtrade.py:369: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [31]:
cols=['reporterDesc','refYear','flowDesc','partnerDesc','rank_reporter','rank_partner','primaryValueFormated', 'perc_year_flow']
show_ony_top=5
top_partners[top_partners.rank_reporter<=show_ony_top].sort_values(by=['refYear','flowDesc','primaryValue'], ascending=[True,True,False])[cols].style.format({'perc_year_flow': '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,partnerDesc,rank_reporter,rank_partner,primaryValueFormated,perc_year_flow
23,China,2011,Import,Angola,1,1,"24,922,180,492.00",37.64%
170,United States of America,2011,Import,Angola,2,1,"13,833,012,040.00",20.89%
117,India,2011,Import,Angola,3,1,"6,005,484,849.00",9.07%
87,"Asia, not elsewhere specified",2011,Import,Angola,4,1,"5,665,315,729.00",8.56%
20,Canada,2011,Import,Angola,5,1,"2,469,608,910.00",3.73%
195,China,2012,Import,Angola,1,1,"33,561,896,917.00",44.46%
330,United States of America,2012,Import,Angola,2,1,"10,030,264,550.00",13.29%
278,India,2012,Import,Angola,3,1,"8,020,442,181.00",10.62%
248,"Asia, not elsewhere specified",2012,Import,Angola,4,1,"5,004,958,562.00",6.63%
282,South Africa,2012,Import,Angola,5,1,"2,802,783,009.10",3.71%


In [38]:
pivoted = top_partners[top_partners.rank_reporter<=show_ony_top].pivot_table(index=['refYear','flowDesc'], columns='reporterDesc', values=['primaryValue','perc_year_flow'])
pivoted.swaplevel(axis=1).sort_index(axis=1, level=0).fillna('')

reporterDesc     Asia, not elsewhere specified                   \
                                perc_year_flow     primaryValue   
refYear flowDesc                                                  
2011    Import                            0.09 5,665,315,729.00   
2012    Import                            0.07 5,004,958,562.00   
2013    Import                            0.05 3,855,306,649.00   
2014    Import                            0.05 2,955,558,824.00   
2015    Import                                                    
2016    Import                            0.05 1,312,764,415.00   
2017    Import                            0.04 1,479,442,599.65   
2018    Import                                                    
2019    Import                                                    
2020    Import                                                    
2021    Import                                                    

reporterDesc             Canada                           China  \
                 perc_year_flow     primaryValue perc_year_flow   
refYear flowDesc                                                  
2011    Import             0.04 2,469,608,910.00           0.38   
2012    Import                                             0.44   
2013    Import                                             0.44   
2014    Import                                             0.48   
2015    Import                                             0.43   
2016    Import                                             0.49   
2017    Import                                             0.56   
2018    Import                                             0.58   
2019    Import                                             0.63   
2020    Import                                             0.62   
2021    Import                                             0.62   

reporterDesc                       France and Monaco                   \
                      primaryValue    perc_year_flow     primaryValue   
refYear flowDesc                                                        
2011    Import   24,922,180,492.00                                      
2012    Import   33,561,896,917.00                                      
2013    Import   31,972,669,347.00                                      
2014    Import   31,106,015,046.00                                      
2015    Import   16,001,611,383.00              0.04 1,610,479,858.00   
2016    Import   13,966,117,080.00                                      
2017    Import   20,698,709,467.00                                      
2018    Import   25,826,510,355.00                                      
2019    Import   23,837,402,142.00                                      
2020    Import   14,757,730,338.00                                      
2021    Import   20,908,231,317.00                                      

reporterDesc              India                   ...   South Africa  \
                 perc_year_flow     primaryValue  ... perc_year_flow   
refYear flowDesc                                  ...                  
2011    Import             0.09 6,005,484,849.00  ...                  
2012    Import             0.11 8,020,442,181.00  ...           0.04   
2013    Import             0.09 6,798,300,153.00  ...                  
2014    Import             0.09 5,641,667,702.00  ...                  
2015    Import             0.09 3,169,598,580.00  ...                  
2016    Import             0.07 2,001,936,255.00  ...           0.05   
2017    Import             0.11 4,118,279,131.90  ...           0.04   
2018    Import             0.10 4,337,925,217.50  ...                  
2019    Import             0.10 3,799,563,165.19  ...                  
2020    Import             0.08 2,022,432,688.74  ...                  
2021    Import             0.07 2,315,947,568.41  ...                  

reporterDesc                               Spain                   \
                     

In [ ]:
top_partners['perc_year_flow'].sum()

1.0

# Exportações

## Principais produtos exportados

In [39]:
show_only_top=50
export_products = comtrade.top_commodities(reporterCode=None,
                                               partnerCode=country_of_interest,
                                               years=years_of_interest, 
                                               cmdCode='AG2',
                                               flowCode='M',  # exports from partners imports
                                           motCode=0,
                                           rank_filter=show_only_top)
export_products.groupby(['refYear','flowDesc','cmdCode','cmdDesc'])['primaryValue'].sum().sort_values(ascending=False).head(10)

/Users/jrc/develop/cipf-comtrade/comtrade.py:369: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


refYear  flowDesc  cmdCode  cmdDesc                                                                                             
2012     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes   71,122,194,738.19
2013     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes   68,806,744,671.76
2011     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes   64,243,246,424.07
2014     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes   62,749,952,891.12
2018     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes   41,004,276,582.51
2019     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral wax

In [ ]:
cols = ['refYear','flowDesc','cmdDesc','cmdCode','reporterDesc', 'rank_cmd','primaryValueFormated',
        comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD,'perc_year_flow']
export_products[export_products['primaryValue']>=1000000].sort_values(['rank_cmd','primaryValue'],ascending=[True, False])[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format,'perc_year_flow': '{:,.2%}'.format})

,refYear,flowDesc,cmdDesc,cmdCode,reporterDesc,rank_cmd,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd,perc_year_flow
193,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,China,1,"20,823,873,963.00",62.20%,68.00%,62.20%
2769,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,India,1,"2,228,646,495.86",6.66%,7.28%,6.66%
3126,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Thailand,1,"1,601,551,152.15",4.78%,5.23%,4.78%
3300,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,United States of America,1,"1,033,920,550.00",3.09%,3.38%,3.09%
2770,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Singapore,1,"737,941,314.25",2.20%,2.41%,2.20%
2965,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Spain,1,"711,309,201.82",2.12%,2.32%,2.12%
192,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Chile,1,"537,652,997.39",1.61%,1.76%,1.61%
975,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Indonesia,1,"459,278,334.00",1.37%,1.50%,1.37%
2772,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,South Africa,1,"341,861,765.06",1.02%,1.12%,1.02%
1332,2021,Import,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",27,Pakistan,1,"322,266,076.65",0.96%,1.05%,0.96%


In [ ]:
export_products[comtrade.PERC_CMD_IN_PARTNER].sum()

0.99991395544394

## Principais produtos exportados e destinos

In [20]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0, 
                                           extra_cols=[],
                                           rank_filter=show_only_top)

In [21]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc',
        comtrade.PERC_PARTNER_IN_CMD, 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER ]

show_only_greater_than = export_products[comtrade.PERC_PARTNER_IN_CMD]>0.01
export_products[show_only_greater_than][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,perc_partner_for_cmd,primaryValueFormated,perc_cmd_for_partner
2216,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,China,63.93%,"20,022,774,116.55",99.82%
4743,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,India,9.28%,"2,907,767,269.85",96.66%
5779,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Thailand,3.21%,"1,004,404,810.25",99.96%
2222,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,Singapore,2.40%,"751,652,390.95",93.65%
4827,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",6,Spain,2.01%,"628,140,238.31",79.11%
1684,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",7,Italy,2.00%,"625,729,106.51",99.00%
5824,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",9,Chile,1.65%,"516,869,632.99",99.73%
2213,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,South Africa,1.56%,"489,628,802.00",94.28%
2343,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",10,Netherlands,1.46%,"456,193,575.29",95.57%
4506,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",12,Indonesia,1.42%,"443,927,852.71",99.93%


## Principais **parceiros** das exportações e produtos exportados

In [14]:
show_only_top = 5
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [15]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner',	'partnerDesc','rank_cmd', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]
export_partners[cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, 
                                                                                       comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
2216,Angola,2021,Export,1,China,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","20,022,774,116.55",99.82%,63.93%
5170,Angola,2021,Export,1,China,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","11,961,705.37",0.06%,25.51%
4696,Angola,2021,Export,1,China,5,Wood and articles of wood; wood charcoal,"11,645,905.65",0.06%,23.44%
890,Angola,2021,Export,1,China,13,"Ores, slag and ash","6,616,586.45",0.03%,66.65%
1100,Angola,2021,Export,1,China,18,Aluminium and articles thereof,"966,966.75",0.00%,17.60%
4743,Angola,2021,Export,2,India,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","2,907,767,269.85",96.66%,9.28%
3057,Angola,2021,Export,2,India,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","100,217,986.40",3.33%,6.52%
1864,Angola,2021,Export,2,India,3,"Ships, boats and floating structures","106,755.20",0.00%,0.01%
1254,Angola,2021,Export,2,India,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","70,361.90",0.00%,0.15%
586,Angola,2021,Export,2,India,73,Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,"46,527.16",0.00%,99.32%


# Importações

## Principais produtos importados

In [22]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [23]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
402,Angola,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","1,886,757,547.56",World,16.58%
230,Angola,2021,Import,2,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","1,809,939,647.27",World,15.91%
126,Angola,2021,Import,3,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","779,123,345.04",World,6.85%
367,Angola,2021,Import,4,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","776,747,585.90",World,6.83%
161,Angola,2021,Import,5,10,Cereals,"584,957,170.07",World,5.14%
27,Angola,2021,Import,6,73,Iron or steel articles,"536,792,046.75",World,4.72%
362,Angola,2021,Import,7,39,Plastics and articles thereof,"405,040,625.98",World,3.56%
240,Angola,2021,Import,8,02,Meat and edible meat offal,"348,250,700.18",World,3.06%
322,Angola,2021,Import,9,30,Pharmaceutical products,"322,068,826.04",World,2.83%
169,Angola,2021,Import,10,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","311,937,457.18",World,2.74%


## Principais produtos importados e origens

In [24]:
show_only_top = 10
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           cmdCode='AG2',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [25]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
3954,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Togo,"687,679,076.53",100.00%,36.45%
13156,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,Netherlands,"318,111,982.79",73.62%,16.86%
13215,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,India,"291,213,622.52",39.05%,15.43%
3776,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",17,Malta,"184,307,230.08",99.00%,9.77%
4069,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",11,Belgium,"113,736,727.71",29.24%,6.03%
7421,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",1,China,"294,737,063.36",17.45%,16.28%
7026,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",2,Portugal,"219,815,557.96",16.19%,12.14%
1351,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",9,Italy,"206,406,850.29",48.33%,11.40%
7219,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",6,United Kingdom,"165,889,250.91",30.75%,9.17%
1111,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",7,United States of America,"159,642,149.65",30.52%,8.82%


### Principal origem das importações

In [32]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [33]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated', comtrade.PERC_PARTNER_IN_CMD]
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated,perc_partner_for_cmd
258,Angola,2021,Import,1,China,"1,688,904,641.34",14.84%
95,Angola,2021,Import,2,Portugal,"1,357,979,633.71",11.93%
323,Angola,2021,Import,3,India,"745,723,527.22",6.55%
319,Angola,2021,Import,4,Togo,"687,689,058.08",6.04%
495,Angola,2021,Import,5,Brazil,"546,044,463.65",4.80%
610,Angola,2021,Import,6,United Kingdom,"539,546,476.65",4.74%
251,Angola,2021,Import,7,United States of America,"523,006,089.83",4.60%
455,Angola,2021,Import,8,Netherlands,"432,100,821.51",3.80%
192,Angola,2021,Import,9,Italy,"427,046,959.44",3.75%
36,Angola,2021,Import,10,France and Monaco,"411,360,213.52",3.62%


## Principais **parceiros** das importações e produtos importados

In [34]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [35]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
7421,Angola,2021,Import,1,China,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","294,737,063.36",17.45%,16.28%
1837,Angola,2021,Import,1,China,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags","207,543,201.07",12.29%,80.04%
5764,Angola,2021,Import,1,China,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","188,027,402.48",11.13%,24.21%
3683,Angola,2021,Import,1,China,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","170,241,136.72",10.08%,21.85%
12591,Angola,2021,Import,1,China,Iron and steel,"94,179,850.47",5.58%,64.83%
5371,Angola,2021,Import,1,China,Plastics and articles thereof,"90,509,512.27",5.36%,22.35%
11724,Angola,2021,Import,1,China,Glass and glassware,"62,248,517.42",3.69%,75.37%
6889,Angola,2021,Import,1,China,Iron or steel articles,"60,344,496.23",3.57%,11.24%
6008,Angola,2021,Import,1,China,Rubber and articles thereof,"58,498,228.59",3.46%,43.76%
11490,Angola,2021,Import,1,China,Pharmaceutical products,"48,373,928.61",2.86%,15.02%


## Análise de simetrias nos dados

In [36]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [83]:
show_only_top = 30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
348,Angola,China,World,2021,Export,"20,058,426,271.91",20058426271.912998,58.19%,All modes of transport,True,All customs procedure codes
211,Angola,India,World,2021,Export,"3,008,390,595.07",3008390595.072000,8.73%,All modes of transport,True,All customs procedure codes
288,Angola,United Arab Emirates,World,2021,Export,"1,399,449,578.20",1399449578.198000,4.06%,All modes of transport,True,All customs procedure codes
165,Angola,Thailand,World,2021,Export,"1,004,833,529.77",1004833529.770000,2.91%,All modes of transport,True,All customs procedure codes
46,Angola,Singapore,World,2021,Export,"802,659,408.76",802659408.761000,2.33%,All modes of transport,True,All customs procedure codes
109,Angola,Spain,World,2021,Export,"793,986,998.99",793986998.994000,2.30%,All modes of transport,True,All customs procedure codes
67,Angola,Italy,World,2021,Export,"632,025,040.54",632025040.539000,1.83%,All modes of transport,True,All customs procedure codes
279,Angola,South Africa,World,2021,Export,"519,358,721.33",519358721.333000,1.51%,All modes of transport,True,All customs procedure codes
70,Angola,Chile,World,2021,Export,"518,264,559.23",518264559.233000,1.50%,All modes of transport,True,All customs procedure codes
63,Angola,Netherlands,World,2021,Export,"477,332,019.79",477332019.785000,1.38%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [82]:
show_only_top=30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     motCode = 0,
                     # customsCode=0,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue','reporterDesc',], ascending=[True,False,True]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 

,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
29,China,Angola,World,2021,Import,"20,908,231,317.00",20908231317.000000,62.45%,All modes of transport,True,All customs procedure codes
324,India,Angola,World,2021,Import,"2,315,947,568.41",2315947568.408000,6.92%,All modes of transport,True,All customs procedure codes
388,United Arab Emirates,Angola,World,2021,Import,"1,685,701,656.52",1685701656.518000,5.04%,All modes of transport,True,All customs procedure codes
385,Thailand,Angola,World,2021,Import,"1,607,756,231.51",1607756231.510000,4.80%,All modes of transport,True,All customs procedure codes
413,United States of America,Angola,World,2021,Import,"1,089,250,402.00",1089250402.000000,3.25%,All modes of transport,True,All customs procedure codes
325,Singapore,Angola,World,2021,Import,"752,550,761.40",752550761.398000,2.25%,All modes of transport,True,All customs procedure codes
355,Spain,Angola,World,2021,Import,"739,484,782.77",739484782.768000,2.21%,All modes of transport,True,All customs procedure codes
5,Belgium,Angola,World,2021,Import,"615,024,126.87",615024126.871000,1.84%,All modes of transport,True,All customs procedure codes
28,Chile,Angola,World,2021,Import,"537,823,552.96",537823552.960000,1.61%,All modes of transport,True,All customs procedure codes
147,Indonesia,Angola,World,2021,Import,"463,466,072.00",463466072.000000,1.38%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [104]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['China']
flowCode = 'M'
flowDesc = comtrade.FLOWS_CODES[flowCode]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}" \
 if flowCode == 'X' else f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}"

cmd_detail = 'AG4'
show_only_top = 10

cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [105]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [106]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    print(phrase)
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''

result

Angola Import from China reported by Angola


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
1021,Angola,China,World,2020,Import,"Rail locomotives; (other than those of heading no. 8601), locomotive tenders","106,385,404.28",106385404.277000,7.42%,All modes of transport,True,All customs procedure codes
1382,Angola,China,World,2020,Import,"New pneumatic tyres, of rubber","50,669,463.76",50669463.757000,3.53%,All modes of transport,True,All customs procedure codes
1087,Angola,China,World,2020,Import,nan,"48,169,359.41",48169359.409000,3.36%,All modes of transport,True,All customs procedure codes
1722,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products, width 600mm or more, clad, plated or coated","40,716,106.16",40716106.155000,2.84%,All modes of transport,True,All customs procedure codes
508,Angola,China,World,2020,Import,"Electric transformers, static converters (e.g. rectifiers) and inductors","39,653,621.92",39653621.919000,2.76%,All modes of transport,True,All customs procedure codes
2340,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products of a width of 600mm or more, hot-rolled, not clad, plated or coated","39,392,570.45",39392570.454000,2.75%,All modes of transport,True,All customs procedure codes
381,Angola,China,World,2020,Import,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other apparatus for the transmission or reception of voice, images or other data (including wired/wireless networks), excluding items of 8443, 8525, 8527, or 8528","36,628,189.17",36628189.175000,2.55%,All modes of transport,True,All customs procedure codes
28,Angola,China,World,2020,Import,"Medicaments; (not goods of heading no. 3002, 3005 or 3006) consisting of mixed or unmixed products for therapeutic or prophylactic use, put up in measured doses (incl. those in the form of transdermal admin. systems) or packed for retail sale","29,087,289.30",29087289.296000,2.03%,All modes of transport,True,All customs procedure codes
2147,Angola,China,World,2020,Import,"Air conditioning machines; comprising a motor driven fan and elements for changing the temperature and humidity, including those machines in which the humidity cannot be separately regulated","22,047,690.54",22047690.539000,1.54%,All modes of transport,True,All customs procedure codes
2671,Angola,China,World,2020,Import,"Clock, watch and similar glasses, glasses for non-corrective or corrective spectacles, curved, bent, hallowed etc, not optically worked; hollow glass spheres and their segments for manufacture","21,833,158.76",21833158.762000,1.52%,All modes of transport,True,All customs procedure codes


In [108]:
# invertemos

print(years_of_interest, partnerA, partnerB)

flowCodeInv = 'X' if flowCode == 'M' else 'M'
flowDesc = comtrade.FLOWS_CODES[flowCodeInv]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}" if flowCodeInv == 'X'\
    else f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}"


print(phrase)

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''
result

2021 24 156
China Export to Angola reported by China


ReadTimeout: HTTPSConnectionPool(host='comtradeapi.un.org', port=443): Read timed out. (read timeout=10)